Author 1: Jeril Bills

Author 2: Aric Bills

Date: September/October 2021

Description: Puts a list of artists into a tournament. For list lengths that are powers of 2, it shuffles a list that can then be used to stage a knockout tournament. For all other list lengths, it sorts artists into groups of 4-8 and schedules a round-robin tournament for each group.



Function to take a list of artists and read the file;
Function takes one argument, filename, which should be a .txt file;
with makes sure that it closes the file when done after opening.;
"r" is to open the file for reading - it's also the default;
strip() removes the white space at the beginning and end of the file;
splitlines() converts the document into a list, with each line of the file as a list item

In [1]:
def read_list(filename):
    with open(filename, "r", encoding = "utf-8") as f:
        return f.read().strip().splitlines()

Function to check whether a number is a power of 2

In [2]:
import math
def is_power2(num):
    power2 = math.log2(num)
    return math.isclose(power2, round(power2))

Function to determine the number of groups;
Why did we do the math this way? The number of groups must be a power of 2, and there must be no more than 8 and no less than 4 per group. So the upper bound (the most possible groups) is listnum/4, and the lower bound (fewest possible groups) is listnum/8. Because 8 is one power of 2 higher than 4, log2 of listnum/8 will always be 1 less than log2 of listnum/4. converting math.log2(listnum/4) automatically returns the power to which we need to raise 2 in order to get the number of groups, since int will round down to the nearest whole integer, and log2 of listnum/4 is our upper bound. 

In [3]:
def num_groups(listnum):
    return 2**int(math.log2(listnum/4))

Function to assign artists to groups;
First we created four variables;
group_num tells us how many groups to create for this list;
per_group tells us the average number of aritsts per group;
artists starts out as an empty list, but will ultimately contain the contents of each group with each group as a list;
start is set at 0 to begin with, since this will be the first start index value from which to start the first group;
iterate through total number of groups; group_num + 1 since the end value doesn't get iterated over, that's where it stops - the reason we did this instead of range(group_num) is because we're multiplying the average number of artists per group by this value each time;
end is set to round. This allows us distribute larger and smaller group numbers evenly throughout;
initial start value is 0 and the initial end value is the average number of artists rounded to the nearest whole number. This will create a list within the artists list of the first however many is the average number of aritsts rounded to the nearest whole number, which will constitute our first group. Since Python stops at the end value without including it, this works out perfectly, since the starting index is 0 and so the last item we want will be the average artist rounded number minus 1;
We then assign the current end index as the start of the next one so we can get the next group of artists.

In [4]:
def assign_groups(artlst):
    group_num = num_groups(len(artlst))
    per_group = len(artlst) / group_num
    artists = []
    start = 0
    for i in range(1, group_num + 1):
        end = round(i * per_group)
        artists.append(artlst[start:end])
        start = end
    return artists

Function to sort a list of artists into groups;
Takes an argument, filename, the name of a text file containing a human list; reads the file and turns it into a python list; shuffles the list; tests whether the list is a power of 2 - if not, break the list into groups such that the number of groups is a power of 2 and the group size is between 4 and 8, and keeping the groups as similar in size as possible, with different-size groups spread evenly throughout; if the list is a power of 2, simply returns the shuffled list.

In [5]:
import random
def group_artists(filename):
    artist_list = read_list(filename)
    random.shuffle(artist_list)
    if is_power2(len(artist_list)) == False:
        return assign_groups(artist_list)
    else: 
        return artist_list

Okay, so now that I've done that, what I want to do now is 
1. print this out in a nice, neat, readable way - for starters. 
2. But then I also want it to create my schedule for me such that every artist battles every other artist once. Ideally, I would like it to do this the way I do it.

Schedule: I think what I want to maybe do is put these into tuples, but I need a way to keep track of who has been picked already. From the list of artists in each group, pick one at random, take the one picked off the list, and then pick another one at random.That's easy. The first day is simple. Do that until there are fewer than 2 artists left. But the next part is trickier. I need a way to pick an artist at random from the list, but then temporarily exclude artists that have already been paired with them, so that the same two artists cannot be paired twice.

My preferred way:

1. Pick an artist at random from the group
2. Pick a second artists at random from the group, different from the first artist and pair the artists together, first artist vs. second artist
3. Keep pairing artists without repeating any until there are fewer than 2 left
4. Repeat until every arist has been paired with every other arist once. Prevent aritsts from being paired with the same artist twice.

One possible (maybe easier?) way:

1. set it up so you have 1 vs. 2, 1 vs. 3, 1 vs. 4, etc., then 2 vs. 3, 2 vs. 4, etc.

Then I could possibly randomize it by shuffling the paired values

Ooh, one way I could achieve the same effect as above would be to do the easier? way, then shuffle pairs, then go through each pair generating a random number (1 or 0 or 1 or 2) like a coinflip to determine whether to keep the pairs in the same order or to flip the order. - Such a program would actually be hella useful once I get to the knockout stage, anyway.

So, the idea is:

1. Pair each arist with each other artists once, 1-2, 1-3, 1-4 ... 2-3, 2-4 ... 
2. shuffle these pairings
3. do a "coin toss" on each pairing to determine whether to keep the order the same or reverse it.

1. use a for loop to iterate through each group, and a nested for loop to iterate through the artists in each group. How do I keep the groups separate, though? And also, is there a more efficient way to do this, since this would be an exponential algorithm, wouldn't it?

Actually - first I have to actually group the artists, no?

pairs_order is a coin-toss function that randomly decides whether to reverse the order of a pairing or leave it the same. First it pairs each artist in the group with each other artist exactly once; then it shuffles the pairings; then for each pairing, at random 50-50, it either leaves the pairing in the same order or reverses the order

print_group prints out the artists in the group in a neat, readable way

print_pairs prints out the matchups in a neat, readable way

print_artists takes a list, puts it in groups and pairings, and then prints out the entire tournament groups and pairings. First it uses the group artists function to put artists into groups; then it uses the print_group function to print the group, followed by the print_pairs function to print the matchups. It does this for each group, printing first the artists in the group, then the matchups.

In [6]:
import itertools
def pairs_order(group):
    pairings = list(itertools.combinations(group, 2))
    random.shuffle(pairings)
    pairings = [(first, second) if random.randrange(2) else (second, first) 
        for (first, second) in pairings]
    return pairings

def print_group(group, group_no):
    print(f"Group {group_no}:")
    print("\n".join(group))

def print_pairs(pairs):
    for (first, second) in pairs:
        print(f"{first} vs. {second}")
    
def pair_artists(filename):
    groups = group_artists(filename)
    for i, group in enumerate(groups, start=1):
        print_group(group,  i)
        print()
        print_pairs(pairs_order(group))
        print()

In [7]:
pair_artists("housetourney.txt")

Group 1:
Deadmau5
David Morales
Danny Rampling
Jonas Blue
Porter Robinson

David Morales vs. Jonas Blue
David Morales vs. Deadmau5
Porter Robinson vs. Danny Rampling
Porter Robinson vs. Jonas Blue
Deadmau5 vs. Jonas Blue
David Morales vs. Porter Robinson
Danny Rampling vs. Deadmau5
Porter Robinson vs. Deadmau5
David Morales vs. Danny Rampling
Danny Rampling vs. Jonas Blue

Group 2:
Yazz
Frankie Knuckles
Tony Humphries
Ron Hardy
Green Velvet

Yazz vs. Tony Humphries
Green Velvet vs. Ron Hardy
Tony Humphries vs. Frankie Knuckles
Yazz vs. Frankie Knuckles
Yazz vs. Ron Hardy
Tony Humphries vs. Ron Hardy
Green Velvet vs. Yazz
Tony Humphries vs. Green Velvet
Frankie Knuckles vs. Green Velvet
Ron Hardy vs. Frankie Knuckles

Group 3:
Wolfgang Gartner
Lady Gaga
Fedde Le Grand
J.M. Silk

Lady Gaga vs. Fedde Le Grand
Wolfgang Gartner vs. Lady Gaga
Fedde Le Grand vs. J.M. Silk
Wolfgang Gartner vs. Fedde Le Grand
Wolfgang Gartner vs. J.M. Silk
Lady Gaga vs. J.M. Silk

Group 4:
Raze
Larry Heard
Jim 

What would be even cooler, but seems much more difficult:
    Figure out how to automatically extract the data from a webpage - make a program that will create my list for me in the first place by taking that data.
    Have it pull up Spotify, search for the artist, and play a song by that artist by going to their spotify artist page and randomly shuffling - so it would be automatic. I would just run my program, and it would go through the tournament on its own and then ask me to pick a winner, then store that value.

In [8]:
def shuffled_list(filename):
    unshuffled_list = read_list(filename)
    shuffled = random.shuffle(unshuffled_list)
    return shuffled

In [9]:
pair_artists("tappatourney.txt")

Group 1:
Free Will
Moving Target
Pieces of a Man
Save the Children
Nothing New
Small Talk at 125th and Lenox

Free Will vs. Moving Target
Pieces of a Man vs. Free Will
Pieces of a Man vs. Nothing New
Pieces of a Man vs. Moving Target
Small Talk at 125th and Lenox vs. Free Will
Save the Children vs. Nothing New
Save the Children vs. Small Talk at 125th and Lenox
Free Will vs. Save the Children
Free Will vs. Nothing New
Small Talk at 125th and Lenox vs. Nothing New
Nothing New vs. Moving Target
Small Talk at 125th and Lenox vs. Moving Target
Pieces of a Man vs. Save the Children
Moving Target vs. Save the Children
Small Talk at 125th and Lenox vs. Pieces of a Man

Group 2:
Spirits
I'm New Here
We're New Again
The First Minute of a New Day
1980
Reflections

1980 vs. Spirits
1980 vs. I'm New Here
We're New Again vs. I'm New Here
The First Minute of a New Day vs. Reflections
We're New Again vs. The First Minute of a New Day
I'm New Here vs. Spirits
The First Minute of a New Day vs. 1980
Ref